In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 110.3 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import pandas as pd
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForSequenceClassification

drive.mount('/content/drive')
project_path = './drive/MyDrive/Colab Notebooks/GPT_community/'

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="single_label_classification")
model

Mounted at /content/drive


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [4]:
rewards_file = os.path.join(project_path, 'data/rewards.csv')
df_rewards = pd.read_csv(rewards_file)
display(df_rewards.tail())

,prompt,response,reward
0,"In the absence of a god, is anything truly goo...","In the absence of a god, is anything truly goo...",1.0
1,"In the absence of a god, is anything truly goo...","In the absence of a god, is anything truly goo...",0.5
2,"In the absence of a god, is anything truly goo...","In the absence of a god, is anything truly goo...",0.0


In [ ]:
"""
Want to produce scalar ratings of prompt responses. Consider Kant says something, then Hume takes that as a prompt and responds.
Kant then produces a rating of Hume's response by combining its sentiment and likelihood of himself saying it (probability per token of
saying what Hume said / probability per token of what Kant would actually have most likely said). These would combine as follows:
- Postive sentiment and high probability = reinforces Kant's weights ('Yeah that was a great thing to say!)
- Positive sentiment and low probability = nothing happens ('Interesting point, but let's agree to disagree.')
- Negative sentiment and high probability = nothing happens (I agree, but this is kinda lame.')
- Negative sentiment and low probability = changes Kant's weights (That is a terrible point, this is better.')
Need some way of defining the difference between low and high probability.
"""